In [1]:
# Run for GPU/CUDA ML, comment out if you dont have it configured.
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [45]:
from numpy import mean
from numpy import std
import numpy as np
from numpy import dstack
import keras.backend as kb
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical, np_utils
from sklearn.preprocessing import LabelEncoder

# load_dataset:
trainxshape = (53466, 50, 6)
trainyshape = (53466, 1)
testxshape = (22911, 50, 6)
testyshape = (22911, 1)

## Load data previously saved as numpy text and reshape it to orginal form
loaded_arr = np.loadtxt("ready_data/x_train.txt")
x_train = loaded_arr.reshape(
    loaded_arr.shape[0], loaded_arr.shape[1] // trainxshape[2], trainxshape[2])

loaded_arr1 = np.loadtxt("ready_data/y_train.txt")
y_train = loaded_arr1

loaded_arr2 = np.loadtxt("ready_data/x_test.txt")
x_test = loaded_arr2.reshape(loaded_arr2.shape[0], loaded_arr2.shape[1] // testxshape[2], testxshape[2])

loaded_arr3 = np.loadtxt("ready_data/y_test.txt")
y_test = loaded_arr3

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# encoder = LabelEncoder()
# encoder.fit(y_train)
# encoded_ytrain = encoder.transform(y_train)
# encoded_ytest = encoder.transform(y_test)

# ohe_ytrain = np_utils.to_categorical(encoded_ytrain, num_classes=7)
# ohe_ytest = np_utils.to_categorical(encoded_ytest, num_classes=7)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(53466, 50, 6)
(53466, 7)
(22904, 50, 6)
(22904, 7)


In [46]:
# new = to_categorical(encoded_ytrain)
# print(new.shape)
# print(new[0])

print(y_test[0])
# print(encoded_ytrain.shape)
# print(encoded_ytrain)
# print(encoded_ytest.shape)
# print(encoded_ytest)
# print(ohe_ytrain.shape)
# print(ohe_ytrain)
# print(ohe_ytest.shape)
# print(ohe_ytest[0])


[0. 0. 0. 0. 0. 0. 1.]


In [47]:
def evaluate_model(x_train, y_train, x_test, y_test, dropout):
    print("start evaluation!")
    LR = 0.0001
    verbose, epochs, batch_size = 1, 10, 64
    # timesteps = window size, #n_features = 6, n_outputs =
    n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps, n_features), dropout=dropout))
    # model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps, n_features), dropout=dropout))
    # model.add(LSTM(10, return_sequences=True))
    # model.add(LSTM(64, return_sequences=True))
    # model.add(LSTM(32, return_sequences=True))
    # model.add(LSTM(100, dropout=dropout))
    # model.add(Dropout(0.5))
    # model.add(Dense(100, activation='relu'))
    # model.add(Dropout(0.3))
    # model.add(Dense(25, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    kb.set_value(model.optimizer.learning_rate, LR)
    model.summary()

    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # save model
    model.save('lstm_model4')
    # evaluate model
    _, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=verbose)
    return accuracy

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [48]:
# run an experiment
def run_experiment(repeats=1):
    dropout = 0.2
    # dropout = [0.0, 0.2, 0.3, 0.4, 0.5]
    scores = list()
    for r in range(repeats):
        score = evaluate_model(x_train, y_train, x_test, y_test, dropout)
        score = score * 100.0
        print('>#%d: %.3f' % (r + 1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)


# run the experiment
run_experiment()


start evaluation!
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               42800     
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 707       
Total params: 43,507
Trainable params: 43,507
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
836/836 [==============================] - 5s 6ms/step - loss: 1.5223 - accuracy: 0.4586
Epoch 2/10
836/836 [==============================] - 5s 6ms/step - loss: 0.8987 - accuracy: 0.7345
Epoch 3/10
836/836 [==============================] - 5s 6ms/step - loss: 0.4007 - accuracy: 0.9108
Epoch 4/10
836/836 [==============================] - 5s 6ms/step - loss: 0.1714 - accuracy: 0.9716
Epoch 5/10
836/836 [==============================] - 5s 6ms/step - loss: 0.0978 - accuracy: 0

## Testing the model

In [49]:
from os.path import join
from os import listdir
from tensorflow.python.keras.backend import set_session
from keras.models import load_model

MODEL_NAME = 'lstm_model4'

# Our samples directory
SAMPLE_PATH = './ready_data'

# session = tf.compat.v1.Session(graph=tf.compat.v1.Graph())

model = load_model(MODEL_NAME)

sample_files = listdir(SAMPLE_PATH)

In [50]:
y_pred = model.predict(x_test)


In [51]:
rounded = np.round(y_pred)

print(y_pred[0])
print(rounded[0])
print(y_test[0])

[2.64585076e-04 4.57587748e-06 2.94613510e-05 6.37756693e-06
 3.49302463e-05 1.39476615e-05 9.99646187e-01]
[0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 1.]


In [52]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def plot_cm(y_true, y_pred, class_names):
  cm = confusion_matrix(y_true, y_pred)
  fig, ax = plt.subplots(figsize=(18, 16)) 
  ax = sns.heatmap(
      cm, 
      annot=True, 
      fmt="d", 
      cmap=sns.diverging_palette(220, 20, n=7),
      ax=ax
  )

  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  ax.set_xticklabels(class_names)
  ax.set_yticklabels(class_names)
  b, t = plt.ylim() # discover the values for bottom and top
  b += 0.5 # Add 0.5 to the bottom
  t -= 0.5 # Subtract 0.5 from the top
  plt.ylim(b, t) # update the ylim(bottom, top) values
  plt.show() # ta-da!

In [53]:
class_names = ["Buddha clap", "Crank left", "Crank right", "Knob left", "Knob right", "Pushback", "Swipe"]
name = np.array(class_names)
print(name)
print(type(name))

['Buddha clap' 'Crank left' 'Crank right' 'Knob left' 'Knob right'
 'Pushback' 'Swipe']
<class 'numpy.ndarray'>


In [60]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(name.reshape(-1, 1))
# y_pred = enc.transform(y_pred)
y_pred = enc.inverse_transform(rounded)
y_test_label = enc.inverse_transform(y_test)

In [61]:
print(y_pred)
print(y_test_label)

[['Swipe']
 ['Swipe']
 ['Swipe']
 ...
 ['Buddha clap']
 ['Buddha clap']
 ['Buddha clap']]
[['Swipe']
 ['Swipe']
 ['Swipe']
 ...
 ['Buddha clap']
 ['Buddha clap']
 ['Buddha clap']]


In [62]:
plot_cm(
  y_test_label,
  rounded,
  name
)

ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets